In [17]:
from crewai import Crew, Task, Agent
from dotenv import load_dotenv
import os
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

In [18]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, MDXSearchTool, FileReadTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path="Hrithik_Koduri_Resume.md")
semantic_search_resume = MDXSearchTool(mdx="Hrithik_Koduri_Resume.md")


Inserting batches in chromadb: 100%|##########| 1/1 [00:01<00:00,  1.75s/it]
2024-12-27 23:08:14,365 - 8198360896 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [19]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [20]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [21]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [22]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

In [23]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [24]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [25]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [26]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


In [27]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

In [28]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.vista.com/Vista/job/Boston-Product-Manager-US-Remote-MA-02111/1245749200/?feedId=281100&utm_source=LinkedInJobPostings&utm_campaign=Vistaprint_LinkedIn',
    'github_url': 'https://github.com/hrithikkoduri/',
    'personal_writeup': """I’m Hrithik Koduri, a recent graduate with dual master’s degrees (MBA and MS-MIS) from the University of Arizona's Eller College of Management. With 4 years of experience in product management, analytical roles, and as a Co-Founder of a startup, I bring a strong blend of technical skills and business acumen to the Product Manager positio"""
}

In [30]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

# Agent: Personal Profiler for Engineers
## Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/hrithikkoduri/) URLs, and personal write-up (I’m Hrithik Koduri, a recent graduate with dual master’s degrees (MBA and MS-MIS) from the University of Arizona's Eller College of Management. With 4 years of experience in product management, analytical roles, and as a Co-Founder of a startup, I bring a strong blend of technical skills and business acumen to the Product Manager positio). Utilize tools to extract and synthesize information from these sources.
# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://jobs.vista.com/Vista/job/Boston-Product-Manager-US-Remote-MA-02111/1245749200/?feedId=281100&utm_source=LinkedInJobPostings&utm_campaign=Vistaprint_LinkedIn) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.


# Agent: Perso

In [1]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

Here's the revised version of Hrithik Koduri's resume tailored to the Product Manager position at Vista:

---
**Hrithik Koduri**  
91 Rainey St, Austin, Texas, 78701  
hrithikkoduri18@gmail.com \| linkedin.com/in/hrithikkoduri/ \| CSPO \| Portfolio \| +1 520-286-2703  

---  

**Objective**  
Energetic and experienced Product Manager with over 4 years of comprehensive experience in ecommerce, specializing in product roadmaps, cross-functional leadership, and enhancing user satisfaction through innovative solutions. Excited to bring a solutions-focused approach and a drive for excellence to the Vista team.

---  

**Professional Experience**

**Lokally** – Tucson, AZ, USA  
*Co-Founder, Product Management*  
Jan 2024 – Jul 2024  
- Launched a zero-to-one product development initiative, managing product strategy, roadmap, and go-to-market strategy, successfully onboarding over 50 local businesses within the first three months.  
- Utilized detailed user research and functional feedback to iterate on product designs, increasing user engagement by 30%.  
- Collaborated closely with engineering teams to oversee microservices architecture and develop UI/UX design prototypes ensuring timely execution and superior product quality.

**Tucson Waters** – Tucson, AZ, USA  
*Product Management Intern*  
May 2023 – Nov 2023  
- Directed the development lifecycle of a mobile application for hazardous event detection which decreased response times by 20%.  
- Developed MVP within projected timeline and budget, resulting in a 30% improvement in feature adoption through strategic user testing.

**Media.net** – Mumbai, India  
*Associate Product Manager*  
Aug 2020 – May 2023  
- Managed a product stream that generated revenues exceeding $4.6M by leading the development and iteration of adaptive advertising solutions.  
- Enhanced bidding strategy microservices which improved real-time bidder win rates by 25%.  

**Fruit Bowl Digital** – Mumbai, India  
*Marketing Analyst Intern*  
Jun 2019 – May 2020  
- Provided market analysis and strategic marketing solutions, boosting traffic and interaction for startup clients by 40%.

---  

**Project Experience - Generative AI**  
- **ConvoPro, DelvInDocs.AI, GitChat, TalkQL:** Developed several AI-powered tools improving documentation clarity, developer productivity, and facilitating context-based interaction enhancing overall business communications.

---  

**Education**  
**University of Arizona – Eller College of Management**  
Master of Science in Management Information Systems (MS-MIS), Aug 2022 - May 2024  
Master of Business Administration (MBA), Aug 2022 - May 2024  

**University of Mumbai**  
Bachelor of Engineering in Information Technology (BEIT), Aug 2016 - May 2020  

---  

**Skills**  
- **Core Skill Set:** Product Roadmapping, Agile Methodologies, Data-Driven Decision Making, Customer-Centric Product Design, Cross-Functional Team Leadership.  
- **Tools:** JIRA, Confluence, AWS, Figma, Salesforce, MS Office Suite.  
- **Programming:** Proficient in Python, JavaScript, SQL; familiar with ReactJS, NodeJS.

---  

**Certifications**  
- Certified Scrum Product Owner (CSPO) – Scrum Alliance  
- Certified in SQL for Data Science – Datacamp  
- Product Analytics – Product School  

---

This resume is more targeted and reflective of the job description provided by Vista, demonstrating Hrithik's readiness to excel in the Product Manager role.

In [2]:
display(Markdown("./interview_materials.md"))

Below is a set of interview questions and talking points specifically designed for Hrithik Koduri's initial interview for the Product Manager position at Vista. These questions and points are crafted to highlight his relevant experiences, technical skills, and overall suitability for the role, aligned with the job requirements provided by Vista.

**Interview Questions:**

1. **Experience with Product Management and Launches**
   - Can you discuss a product you developed from inception to launch? Specifically, detail your role in defining the product strategy, managing the roadmap, and executing the go-to-market plan.

2. **Cross-Functional Team Leadership**
   - Describe an instance where you had to lead a cross-functional team. What challenges did you face and how did you ensure effective collaboration across different departments?

3. **User-Centric Product Development**
   - You've mentioned utilizing user research and feedback to iterate on product designs. Could you share how you gather and integrate customer insights to enhance product performance?

4. **Technical Skills and Project Management**
   - Given your technical background, how have you leveraged your programming and system design skills in product management? Please provide examples of how you've used these skills in your projects.

5. **Agile and Scrum Methodologies**
   - As a Certified Scrum Product Owner, how do you integrate Agile principles in your product management processes? Could you walk us through a typical sprint planning session you've conducted?

6. **Analytical Abilities and Data-Driven Decision Making**
   - Can you discuss a situation where you had to make a quick, effective decision based on analytics? What data sources did you rely on, and what was the outcome?

7. **Experience with E-commerce and Digital Products**
   - Considering your background with e-commerce platforms, what unique approaches did you take to optimize product features and enhance user engagement online?

8. **Handling Complex and Challenging Situations**
   - Share an example of a complex problem you faced while managing a product and the innovative solution you implemented to resolve it.

**Talking Points:**

- Highlight your entrepreneurial experience through your role as a Co-Founder and how it has equipped you with a unique perspective on product management.
- Emphasize your technical prowess with programming languages and AI projects, which could be beneficial for the technology-driven products at Vista.
- Discuss your approach to managing product life cycles and your strategies for ensuring the success of a product in a competitive market.
- Detail your success in creating products that foster user engagement and satisfaction, aligning with Vista’s customer-centric product development approach.
- Touch on your certifications and continuous learning in product management and related fields, exemplifying your commitment to professional growth.

These questions and points aim to provide a comprehensive basis for Hrithik Koduri to prepare for his interview, ensuring that his skills, experiences, and potential contributions to Vista are effectively communicated.